<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/philipp/notebooks/ExtraTask2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extra Task 2: Continuous Text Evaluation

## Imports and Setup

In [1]:
!pip install datasets


import json
import time
import re
from google.colab import userdata
import google.generativeai as genai
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
url1 = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/qa_dataset.json'
data = pd.read_json(url1)

data.head()

,type,question,options,intended_answer,context,timestamp
0,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Absolutely, I'm totally okay with data process...",2024-12-26 13:54:58.593
1,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Yeah, you have my full consent for data proces...",2024-12-26 13:54:58.593
2,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Oh, data processing consent? Yes, you can go r...",2024-12-26 13:54:58.593
3,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,Regarding your request for data processing con...,2024-12-26 13:54:58.593
4,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Yes, please proceed with the data processing, ...",2024-12-26 13:54:58.593


In [3]:
url2 = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/philipp/qa_dataset_continuous.json' # Todo: Change branch
continuous_data = pd.read_json(url2)

continuous_data.head()

,text,questions
0,"In regards to the customer group, it's importa...","[{'question': 'Customer group', 'intended_answ..."
1,"Nope, definitely not, please don’t send me any...",[{'question': 'Would you like to receive marke...
2,"And finally, and this is something I always ap...",[{'question': 'How could this app better suppo...
3,"Also, sometimes I think it’s just a tad slow t...",[{'question': 'How could we improve your exper...
4,"Finally, something I'm struggling with quite a...",[{'question': 'What specific challenges are yo...


In [4]:
qa_data = continuous_data.rename(columns={'text': 'context'})
qa_data['questions'] = qa_data['questions'].apply(lambda x: [q['question'] for q in x] if isinstance(x, list) else [])

qa_data.head()

,context,questions
0,"In regards to the customer group, it's importa...","[Customer group, What is the expected completi..."
1,"Nope, definitely not, please don’t send me any...",[Would you like to receive marketing informati...
2,"And finally, and this is something I always ap...",[How could this app better support your daily ...
3,"Also, sometimes I think it’s just a tad slow t...",[How could we improve your experience using th...
4,"Finally, something I'm struggling with quite a...",[What specific challenges are you currently fa...


## Clustering approach

**Problem**: Question could be answered in multiple sentences and we don't know which sentence answers which question

**Idea**: Split sentences and cluster them in 3 clusters (initial assumption that a sentence contains exactly 3 questions)




# Tried out different clustering approaches like K-Means, Vectorizing, Semantic clustering with Transformer-models like BERT or Sentence-Bert for sentence embedding, DBSCAN

In [36]:
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [37]:
for text in continuous_data['text']:
    sentences = [sentence.strip(' "') for sentence in re.split(r'[.!?]', text) if sentence]
    sentence_embeddings = model.encode(sentences)
    clustering = AgglomerativeClustering(n_clusters=3, metric='cosine', linkage='average')
    labels = clustering.fit_predict(sentence_embeddings)

    clusters = {}
    for sentence, label in zip(sentences, labels):
        clusters.setdefault(label, []).append(sentence)

    for cluster_id, grouped_sentences in clusters.items():
        print(f"Cluster {cluster_id}:")
        for sentence in grouped_sentences:
            print(f"  - {sentence}")
    print("\n\n")

Cluster 2:
  - In regards to the customer group, it's important to note that we are talking about R&D, that is, the research and development section of the organisation
Cluster 1:
  - Let me see, my best guess is that it'll be done in approximately three months, that's the current timeline
Cluster 0:
  - My job duties aren't straightforward, and I tend to wear many hats, so if I had to label myself, I suppose I would say my primary role in the organization is Other



Cluster 2:
  - Nope, definitely not, please don’t send me any promotional emails, I’d really appreciate it
Cluster 1:
  - Yes, I can do that, no problem at all, my phone number is +1-147-649-9205, I look forward to hearing from you
Cluster 0:
  - Okay, so after looking over the choices, and thinking about it a little bit, I'd have to say that my primary goal falls under the category of 'Other', since none of the given options are quite right for me



Cluster 0:
  - And finally, and this is something I always appreciate i

In [38]:
for text in continuous_data['text']:
    sentences = [sentence.strip(' "') for sentence in re.split(r'[.!?]', text) if sentence]
    sentence_embeddings = model.encode(sentences)

    kmeans = KMeans(n_clusters=3)
    labels = kmeans.fit_predict(sentence_embeddings)

    clusters = {}
    for sentence, label in zip(sentences, labels):
        clusters.setdefault(label, []).append(sentence)

    for cluster_id, grouped_sentences in clusters.items():
        print(f"Cluster {cluster_id}:")
        for sentence in grouped_sentences:
            print(f"  - {sentence}")
    print("\n\n")

Cluster 0:
  - In regards to the customer group, it's important to note that we are talking about R&D, that is, the research and development section of the organisation
Cluster 1:
  - Let me see, my best guess is that it'll be done in approximately three months, that's the current timeline
Cluster 2:
  - My job duties aren't straightforward, and I tend to wear many hats, so if I had to label myself, I suppose I would say my primary role in the organization is Other



Cluster 0:
  - Nope, definitely not, please don’t send me any promotional emails, I’d really appreciate it
Cluster 2:
  - Yes, I can do that, no problem at all, my phone number is +1-147-649-9205, I look forward to hearing from you
Cluster 1:
  - Okay, so after looking over the choices, and thinking about it a little bit, I'd have to say that my primary goal falls under the category of 'Other', since none of the given options are quite right for me



Cluster 1:
  - And finally, and this is something I always appreciate i

KeyboardInterrupt: 

# Too many limitations -->
## Better approach: Use Q&A model

In [8]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [9]:
all_results = []

for idx, row in qa_data.iterrows():
    context = row['context']
    questions = row['questions']

    results = []

    for question in questions:
        result = qa_pipeline(question=question, context=context)

        if result['score'] > 0.5:
            results.append({
                "question": question,
                "answer": result['answer'],
                "confidence": result['score']
            })
        else:
            results.append({
                "question": question,
                "answer": None,
                "confidence": result['score']
            })

    all_results.append({
        "context": context,
        "questions_answers": results
    })

In [10]:
for i in range(3):
    for res in all_results[i]['questions_answers']:
        print(f"Question: {res['question']}")
        print(f"Answer: {res['answer']}")
        print(f"Confidence: {res['confidence']:.2f}\n")
    print("-------------------------------")
    print()

Question: Customer group
Answer: None
Confidence: 0.06

Question: What is the expected completion date?
Answer: three months
Confidence: 0.57

Question: What is your primary role in your organization?
Answer: Other
Confidence: 0.96

-------------------------------

Question: Would you like to receive marketing information from via e-mail?
Answer: None
Confidence: 0.31

Question: To ensure we can reach you promptly, could you please provide a phone number?
Answer: None
Confidence: 0.28

Question: Which of these best describes your primary goal for using this app today?
Answer: None
Confidence: 0.44

-------------------------------

Question: How could this app better support your daily work tasks?
Answer: None
Confidence: 0.11

Question: What is your business phone number?
Answer: +49-545-207-2136
Confidence: 0.86

Question: Which of these project management methodologies are you most familiar with?
Answer: None
Confidence: 0.46

-------------------------------



# Summarization of querstions

In [39]:
# API Setup
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-1.5-flash')

In [18]:
def generate_text(prompt):
    try:
        response = model.generate_content(
            prompt,
            generation_config = genai.GenerationConfig(
                temperature=2.0,
            )
        )
        time.sleep(5) # Avoid exceeding API limits
        return response.text.strip()
    except Exception as e:
        exit("Error during API call: ", e)

In [19]:
def rephrase_question(text):
    prompt = f"""
        Reformulate the following statement into a clear, concise, and
        grammatically correct question that directly addresses the user. For
        example, 'Do you consent to data processing?' or 'What kind of follow-up
        would you prefer?'. If the text is already a question, preserve its
        meaning and intent without altering the content or facts. The statement
        is: '{text}'. Return only the reformulated question, without any
        additional explanations, comments, or text.
    """
    return generate_text(prompt)




In [20]:
dfs = list()

for q in range(1, 6):
    url = f'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/questionnaires/questionnaire{q}.json'
    temp_df = pd.read_json(url)

    # Unpack options into an array
    temp_df['options'] = temp_df['options'].apply(lambda x: [option['option'] for option in x])

    # Remove options for specific question types
    # because irrelevant or do not contribute meaningfully to the dataset
    temp_df.loc[temp_df['type'].isin(['TEXT', 'NUMBER', 'DATE']), 'options'] = None

    temp_df['questionnaire'] = f"Questionnaire {q}"

    dfs.append(temp_df)

df = pd.concat(dfs, ignore_index=True)

df['rephrased_question'] = df['question'].apply(rephrase_question)
df = df[['questionnaire', 'rephrased_question']]

df.head()

,questionnaire,rephrased_question
0,Questionnaire 1,Do you consent to data processing?
1,Questionnaire 1,Which customer group?
2,Questionnaire 1,Which products are you interested in?
3,Questionnaire 1,What kind of follow-up is planned?
4,Questionnaire 1,Who should I copy on the follow-up?


In [12]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [33]:
def test_combine(text):
    prompt = f"""
        Combine following questions into a cohesive text with a lenght of at least 150 tokens: {text}
    """
    return generate_text(prompt)

In [35]:
def summarize_questions(questions):
    text = test_combine(questions)
    summary = summarizer(text, max_length=30, min_length=10, do_sample=False)
    return summary[0]['summary_text']


grouped_questions = df.groupby("questionnaire")["rephrased_question"].apply(lambda x: " ".join(x)).reset_index()

grouped_questions["summary"] = grouped_questions["rephrased_question"].apply(summarize_questions)

for idx, row in grouped_questions.iterrows():
    print(f"Questionnaire: {row['questionnaire']}")
    print(f"Summary: {row['summary']}\n")

Questionnaire: Questionnaire 1
Summary: Do you consent to the processing of your data as part of our follow-up process? This data processing will allow us to better understand

Questionnaire: Questionnaire 2
Summary: Would you be interested in receiving occasional marketing emails from us? We understand the importance of respecting your inbox and promise to keep these emails informative

Questionnaire: Questionnaire 3
Summary: To better understand your needs and tailor our services effectively, we need a little more information about your company. Could you please tell us what

Questionnaire: Questionnaire 4
Summary: To best assist you, I need some information. English is my default, but I'm proficient in many others. Knowing your preference ensures

Questionnaire: Questionnaire 5
Summary: We'd appreciate it if you could take a few moments to answer some brief questions. Could you describe the type of customer your organization represents

